In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_PM_ext

No dicts found; please check load_dicts...


In [2]:
xi_list = GLS_Apr_weekday_PM_ext.xi_list
P = GLS_Apr_weekday_PM_ext.P
L = GLS_Apr_weekday_PM_ext.L  # dimension of xi

550

In [3]:
xi_list

550-element Array{Any,1}:
   4.15405e-5 
   1.52965e-9 
   1.52977e-9 
 480.571      
   1.25646e-9 
   3.5177e-5  
   5.22872e-7 
   1.4927e-9  
   1.23142e-9 
   6.14785e-8 
   4.90978e-5 
   5.24944e-7 
   1.49272e-9 
   ⋮          
   1.00895e-5 
   1.60462e-9 
   9.62445e-6 
   1.03966e-5 
   9.11491e-6 
  62.1084     
   5.4964e-6  
  30.5506     
   4.52269e-11
   4.52269e-11
   4.52296e-11
   2.10319e-9 

In [4]:
size(P, 1), size(P, 2)

(110,550)

In [5]:
using JuMP

mGLSJulia = Model()

@defVar(mGLSJulia, lam[1:size(P,1)] >= 0)

@defVar(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @addConstraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @addNLConstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @addNLConstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@setNLObjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   241900
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:   181500

Exception of type: TOO_FEW_DOF in file "IpIpoptApplication.cpp" at line 887:
 Exception message: status != TOO_FEW_DEGREES_OF_FREEDOM evaluated false: Too few degrees of freedom (rethrown)!

EXIT: Problem has too few degrees of freedom.


:Error

In [6]:
getValue(lam)

110-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [7]:
getObjectiveValue(mGLSJulia)

0.0

In [8]:
GLS_Apr_weekday_PM_ext.saveDemandVec(getValue(lam))

LoadError: PyError (:PyObject_Call) <type 'exceptions.IndexError'>
IndexError('index 110 is out of bounds for axis 0 with size 110',)
  File "GLS_Apr_weekday_PM_ext.py", line 145, in saveDemandVec
    lam_dict[key] = lam_list[idx]

while loading In[8], in expression starting on line 1